## Proposal_Group7

In [5]:
library(cowplot)
library(datateachr)
library(digest)
library(infer)
library(repr)
library(taxyvr)
library(tidyverse)

In [6]:
salary_dataset <- read.csv("Salary_Data_Based_country_and_race.csv")

In [8]:
head(salary_dataset)

,X,Age,Gender,Education.Level,Job.Title,Years.of.Experience,Salary,Country,Race
,<int>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,0,32,Male,Bachelor's,Software Engineer,5,90000,UK,White
2,1,28,Female,Master's,Data Analyst,3,65000,USA,Hispanic
3,2,45,Male,PhD,Senior Manager,15,150000,Canada,White
4,3,36,Female,Bachelor's,Sales Associate,7,60000,USA,Hispanic
5,4,52,Male,Master's,Director,20,200000,USA,Asian
6,5,29,Male,Bachelor's,Marketing Analyst,2,55000,USA,Hispanic


## Introduction

## Prelimary Results

## Methods: Plan

## References